<a href="https://colab.research.google.com/github/linhtoroi/hihi/blob/main/DS_TEST_PRACTICE9_T3_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ĐÁNH GIÁ NĂNG LỰC KỸ SƯ KHOA HỌC DỮ LIỆU THÁNG 03 - 2022
#### BÀI THỰC HÀNH
---

Ngày test: 25/03/2022
Thời gian làm bài: 85 phút, từ 10:00 đến 11:25
(+ 5 phút thiết lập môi trường và nộp bài)


**Quy định:**
- Không trao đổi trong suốt quá trình làm bài
- Record màn hình làm bài và share lại cho quanvh8 sau khi kết thúc thời gian test
- Được sử dụng tài liệu và tra cứu internet
- Sau khi hoàn thành, tải notebook, gửi về email: quanvh8@viettel.com.vn

**Hướng dẫn làm bài:**

* Chạy toàn bộ lệnh trong phần 0. CHUẨN BỊ MÔI TRƯỜNG
* Làm theo hướng dẫn tại các câu và CHỈ SỬA các đoạn code nằm giữa các thẻ:

      # YOUR CODE START
      # / END YOUR CODE

*Lưu ý: hầu hết câu hỏi không có tính tuần tự (không cần hoàn thành câu trước để làm câu sau)*

*Đề thi có một số testcase hỗ trợ DS kiểm tra kết quả, nhưng đây không phải điểm số. Điểm số được giám khảo chấm và gửi lại sau.*



---

### 0. CÀI ĐẶT MÔI TRƯỜNG
Guide: Chạy toàn bộ các lệnh trong phần này <br>
KHÔNG SỬA CÁC CÂU LỆNH TRONG PHẦN NÀY

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 27.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=3341b7f7c500f9c04e5a3979696d793c204a3a10294475a54203e05eef2aa0e9
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import gzip
import shutil
import requests 
import seaborn as sns
import matplotlib.pyplot as plt

import pyspark
from pyspark.sql import functions as F
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [ ]:
def data_downloader(file_name):
    file_url = 'https://datasets.imdbws.com/'+file_name
    file_path = "/content/"+file_name
    
    r = requests.get(file_url, stream = True, verify=False) 
    
    with open( file_path, "wb") as file: 
        for block in r.iter_content(chunk_size = 1024*10): 
            if block: 
                file.write(block)
    print("Loaded "+file_name+" to local path: "+file_path)

# TEST CASE
class TEST():
    def test_1(df):
        tc1 = (df.columns ==['tconst', 'averageRating', 'numVotes'])
        tc2 = (df.count() >= 1227518)
        if (tc1 and tc2): print("PASS")
        else: print('FAIL')

    def test_2(df):
        tc1 = (df.columns == ['titleId','ordering','title','region','language','types','attributes','isOriginalTitle'])
        tc2 = (df.count() >= 31450282)
        if (tc1 and tc2): print("PASS")
        else: print('FAIL')

    def iia_schema(df_pred):
      tc1 = (df_pred.columns == ['tconst', 'pred'])
      tc2 = (df_pred.count() == 53956 )
      if tc1 and tc2 :
        print("PASS: dataframe đầu ra đúng định dạng")
      else:
        print("FAIL: dataframe đầu ra không đúng định dạng, đ/c kiểm tra lại")
      
      tc3 = (df_pred.join(test_ratings, on = 'tconst').count() == 53956)
      
      if tc3 :
        print("PASS: dataframe đầu ra đúng số lượng bản ghi")
      else:
        print("FAIL: dataframe đầu ra không đúng số lượng bản ghi, đ/c kiểm tra lại")
        
    def mae_iia(df_pred):
      result = df_pred.join(test_ratings, on = 'tconst')
      result = result.withColumn('ae', F.abs(F.col("pred")-F.col("averageRating")))
      result.selectExpr("avg(ae) AS mae").show()

    def iib_schema(df_pred):
      tc1 = ( list(df_pred.columns) == ['tconst', 'pred'])
      tc2 = (df_pred.count()['tconst'] == 104488 )
      if tc1 and tc2 :
        print("PASS: dataframe đầu ra đúng định dạng")
      else:
        print("FAIL: dataframe đầu ra không đúng định dạng, đ/c kiểm tra lại")
      
      tc3 = ( pd.merge(df_pred, test, how='inner', on = 'tconst')['tconst'].count() == 104488)
      
      if tc3 :
        print("PASS: dataframe đầu ra đúng số lượng bản ghi")
      else:
        print("FAIL: dataframe đầu ra không đúng số lượng bản ghi, đ/c kiểm tra lại")
        
    def score_iib(df_pred):
      eval = pd.merge(df_pred, test, how='inner', on = 'tconst' )
      print( roc_auc_score( eval['isAdult'], eval['pred'] ))
    

In [ ]:
spark = pyspark.sql.SparkSession \
    .builder \
    .appName("DS_test") \
    .getOrCreate()

#### DATA: Bộ IMDb

Thông tin bộ dữ liệu IMDb: https://www.imdb.com/interfaces/

Each dataset is contained in a gzipped, tab-separated-values (TSV) formatted file in the UTF-8 character set. The first line in each file contains headers that describe what is in each column. A ‘\N’ is used to denote that a particular field is missing or null for that title/name

![Alt text](https://relational.fit.cvut.cz/assets/img/datasets-generated/imdb_ijs.svg "Schema bộ dữ liệu")

In [ ]:
# Tải dữ liệu về
file_list = ["title.akas.tsv.gz", "title.ratings.tsv.gz", 
             "title.basics.tsv.gz", "title.basics.tsv.gz", "title.crew.tsv.gz", "title.principals.tsv.gz",
            "name.basics.tsv.gz" ]
      
for i in file_list:
    data_downloader(i)

/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datasets.imdbws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Loaded title.akas.tsv.gz to local path: /content/title.akas.tsv.gz
Loaded title.ratings.tsv.gz to local path: /content/title.ratings.tsv.gz


/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datasets.imdbws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datasets.imdbws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Loaded title.basics.tsv.gz to local path: /content/title.basics.tsv.gz


/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datasets.imdbws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

Loaded title.crew.tsv.gz to local path: /content/title.crew.tsv.gz


/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datasets.imdbws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Loaded title.principals.tsv.gz to local path: /content/title.principals.tsv.gz


/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datasets.imdbws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Loaded name.basics.tsv.gz to local path: /content/name.basics.tsv.gz


## PHẦN I. CHUNG
[ 10 điểm ] <br>
Guide: Sử dụng pyspark hoặc spark-SQL thực hiện tất cả các câu thuộc phần này.

**Câu 1**. [1đ] Đọc các file vào pyspark dataframe, biết rằng các cột trong các file dữ liệu đều phân cách nhau bởi dấu tab (\t). Lưu tương ứng các dataframe với các file dữ liệu:
- df_rating : "title.ratings.tsv.gz"
- df_akas  :"title.akas.tsv.gz"
- df_basics  :"title.basics.tsv.gz"
- df_crew  :"title.crew.tsv.gz"
- df_principals :"title.principals.tsv.gz"
- df_person  : "name.basics.tsv.gz"

(Xem mô tả quan trọng của dữ liệu phía trên)

In [ ]:
df_ratings = spark.read.options(header = 'True', inferSchema = 'True', delimiter='\t').csv('/content/title.ratings.tsv.gz')

df_akas= spark.read.options(header = 'True', inferSchema = 'True', delimiter='\t').csv('/content/title.akas.tsv.gz')

df_basics = spark.read.options(header = 'True', inferSchema = 'True', delimiter='\t').csv('/content/title.basics.tsv.gz')

df_crew = spark.read.options(header = 'True', inferSchema = 'True', delimiter='\t').csv('/content/title.crew.tsv.gz')

df_principals = spark.read.options(header = 'True', inferSchema = 'True', delimiter='\t').csv('/content/title.principals.tsv.gz')

df_person = spark.read.options(header = 'True', inferSchema = 'True', delimiter='\t').csv('/content/name.basics.tsv.gz')

In [ ]:
TEST.test_1(df_ratings)
TEST.test_2(df_akas)

PASS
PASS


**Câu 2**. [1đ] Tìm top 3 ngôn ngữ có nhiều nội dung ( bao gồm tất cả nội dung movie, tv show,...) nhất trong bộ dataset này

In [ ]:
# YOUR CODE START 
df_akas.filter(F.col('language')!='\\N').groupby('language').agg(F.count('titleId').alias('cnt')).orderBy(F.col('cnt').desc()).show(3)
# / END YOUR CODE

+--------+-------+
|language|    cnt|
+--------+-------+
|      ja|4121847|
|      fr|4089140|
|      hi|4059953|
+--------+-------+
only showing top 3 rows



**Câu 3**. [2đ]
Trong các bộ phim Mỹ (region="US", titleType="movie") được công chiếu giai đoạn 2010-2015, tìm top 5 thể loại phim có bình quân rating điểm cao nhất.

In [ ]:
# YOUR CODE START 
df_basics.filter((F.col('startYear')>=2010)&(F.col('startYear')<=2015)&(F.col('titleType')=='movie'))\
.join(df_akas.filter(F.col('region')=='US'), df_basics.tconst==df_akas.titleId)\
.join(df_ratings, on='tconst').groupby('genres').agg(F.avg('averageRating').alias('avg')).orderBy(F.col('avg').desc()).show(5)
# / END YOUR CODE

+--------------------+---+
|              genres|avg|
+--------------------+---+
|Biography,Documen...|9.1|
|Documentary,News,...|8.9|
|           Game-Show|8.9|
|Action,Fantasy,My...|8.8|
|Music,Mystery,Thr...|8.7|
+--------------------+---+
only showing top 5 rows



**Câu 4**. [3đ] Bạn đặt ra một giả thuyết rằng một đạo diễn có xu hướng hợp tác với các diễn viên thân thuộc với họ, hơn là liên tục tìm kiếm các diễn viên mới cho các phim khác nhau. <br>
Hãy lựa chọn các metrics phù hợp, phân tích từ bộ dữ liệu thực tế và rút ra kết luận cho giả thuyết trên có đúng hay không? <br>
( Phân tích trên thể loại phim titleType="movie", region="US",  thời gian công chiếu trong khoảng 2000-2020)

In [ ]:
df_basics.show(2)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|           genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|  Animation,Short|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
only showing top 2 rows



In [ ]:
df_akas.show(2)

+---------+--------+----------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|     title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+----------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|Карменсіта|    UA|      \N|imdbDisplay|           \N|              0|
|tt0000001|       2|Carmencita|    DE|      \N|         \N|literal title|              0|
+---------+--------+----------+------+--------+-----------+-------------+---------------+
only showing top 2 rows



In [ ]:
df_ratings.show(2)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    1965|
|tt0000002|          5.8|     262|
+---------+-------------+--------+
only showing top 2 rows



In [ ]:
df_principals.show(5)

+---------+--------+---------+---------------+--------------------+----------+
|   tconst|ordering|   nconst|       category|                 job|characters|
+---------+--------+---------+---------------+--------------------+----------+
|tt0000001|       1|nm1588970|           self|                  \N|  ["Self"]|
|tt0000001|       2|nm0005690|       director|                  \N|        \N|
|tt0000001|       3|nm0374658|cinematographer|director of photo...|        \N|
|tt0000002|       1|nm0721526|       director|                  \N|        \N|
|tt0000002|       2|nm1335271|       composer|                  \N|        \N|
+---------+--------+---------+---------------+--------------------+----------+
only showing top 5 rows



In [ ]:
df_person.show(2)

+---------+-------------+---------+---------+--------------------+--------------------+
|   nconst|  primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+-------------+---------+---------+--------------------+--------------------+
|nm0000001| Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0045537,tt00531...|
|nm0000002|Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0037382,tt01170...|
+---------+-------------+---------+---------+--------------------+--------------------+
only showing top 2 rows



In [ ]:
df_crew.show(2)

+---------+---------+-------+
|   tconst|directors|writers|
+---------+---------+-------+
|tt0000001|nm0005690|     \N|
|tt0000002|nm0721526|     \N|
+---------+---------+-------+
only showing top 2 rows



In [ ]:
df_crew.join(df_principals.filter((F.col('category')=='actor')|(F.col('category')=='actress')), on='tconst').groupby(['directors', 'nconst']).count().show(5)

+-------------------+---------+-----+
|          directors|   nconst|count|
+-------------------+---------+-----+
|          nm0887428|nm0039276|    1|
|nm0002177,nm0001124|nm2328128|    2|
|          nm0000122|nm0701012|   31|
|          nm0551375|nm0175069|    1|
|nm0285684,nm0692105|nm0717066|    1|
+-------------------+---------+-----+
only showing top 5 rows



In [ ]:
%%time
# YOUR CODE START 

"""Nhận xét: nếu các đạo diễn có xu hướng lựa chọn diễn viên thân thuộc là đúng, sẽ có rất nhiều bộ phim có sự hợp tác của họ
=> Vẽ phân phối số lần hợp tác sẽ đánh giá được giả thiết
"""
# Lấy tập data theo yêu cầu đề bài
movie = df_basics.filter((F.col("titleType")=="movie") &  F.col("startYear").between(2010, 2015) )
us = df_akas.filter(F.col("region")=="US").selectExpr('titleId AS tconst')

principals = df_principals.filter( F.col('category').isin('actor', 'actress', 'director') )\
                .join(movie, on='tconst').join(us, on='tconst')

# Đếm số lần hợp tác
actor = principals.filter(F.col('category').isin('actor', 'actress')).withColumn('actor', F.col('nconst'))
director = principals.filter(F.col('category')=='director').withColumn('director', F.col('nconst'))

colaboration = actor.join(director, on='tconst').groupby('actor', 'director').agg(F.count('tconst').alias('movie_num'))\
                .withColumn('is_1time', F.when(F.col('movie_num')==1, 1).otherwise(0))

his = colaboration.groupby('is_1time').count()
print('Số bộ phim: ', actor.join(director, on='tconst').select(F.countDistinct('tconst')).collect())
print('Số lần hợp tác:')
his.show()
# / END YOUR CODE

Số bộ phim:  [Row(count(DISTINCT tconst)=29891)]
Số lần hợp tác:
+--------+------+
|is_1time| count|
+--------+------+
|       1|119208|
|       0| 12741|
+--------+------+

CPU times: user 1.82 s, sys: 192 ms, total: 2.01 s
Wall time: 6min 35s


In [ ]:
your_conclusion = ('''
# YOUR CONCLUSION HERE


''')

**Câu 5**. [3đ] Một giải thưởng về phim (titleType = 'movie') được trao cho các đạo diễn. Danh sách đề cử là các đạo diễn thỏa mãn các điều kiện: <br>
1. Đạo diễn vẫn còn sống
2. Có trên 5 bộ phim hay (phim hay là phim được chấm điểm averageRating >7.5).
3. Tỷ trọng phim tệ (phim tệ là phim có averageRating <= 4) chiếm không quá 20% tổng số phim của đạo diễn đó và không quá 10 bộ phim.

Trong đó, averageRating chỉ được tính khi có trên 200 phiếu bầu. <br>
Giải thưởng được trao cho 3 trong số các đạo diễn thuộc danh sách đề cử. <br>
Đặt p (float) là xác suất để đạo diễn trúng cử đồng thời có kiêm biên kịch (writer, xác định bởi primaryProfession).  <br>
Hãy tính p.

In [ ]:
# YOUR CODE START 
alive_directors=df_person.filter((F.col('deathYear')=='\\N')&(df_person.primaryProfession.contains('director')))
count_great_movies=df_principals.filter(F.col('category')=='director').join(df_ratings, on='tconst').filter(F.col('numVotes')>200).filter(F.col('averageRating')>7.5).groupby('nconst').agg(F.count('tconst').alias('cnt'))
has_great_movies=alive_directors.join(count_great_movies, on='nconst').filter(F.col('cnt')>5)

# / END YOUR CODE

In [ ]:
from pyspark.sql.window import Window

windowSpec  = Window.partitionBy("nconst")

ratio_bad_movies = df_principals.filter(F.col('category')=='director').join(df_ratings, on='tconst').filter(F.col('numVotes')>200).withColumn('ratio', F.when(F.col('averageRating')<=4, F.count(F.when(F.col('averageRating')<=4, True))\
                                                                                                             .over(windowSpec)).otherwise(0)/F.count('tconst').over(windowSpec))
greatest_directors = has_great_movies.join(ratio_bad_movies, on='nconst').filter(F.col('ratio')<=0.2)

In [ ]:
is_writer = greatest_directors.filter(F.col('primaryProfession').contains('writer')).count()
all = greatest_directors.count()

In [ ]:
import math
p=math.comb(is_writer,3)/math.comb(all,3)
p

0.03516829994930383

In [ ]:
math.comb(all,3)

69293710549470

In [ ]:
%%time
# YOUR CODE START 
film = df_basics.filter(F.col("titleType")=="movie").select("tconst")
director = df_person.filter(F.col("deathYear").isNull()).select('nconst', 'primaryProfession')
ratings = df_ratings.filter(F.col("numVotes")>200)

principals = df_principals.filter( F.col('category')=='director' ).select('nconst', 'tconst')

matched = ratings.join(film, on='tconst')\
                .join(principals, on='tconst')\
                .join(director, on='nconst')\
                .withColumn('is_bad', F.when(F.col('averageRating') <=4, 1).otherwise(0))\
                .withColumn('is_good', F.when(F.col('averageRating') >7.5, 1).otherwise(0))\
                .withColumn('is_writer', F.when(F.col('primaryProfession').contains('writer'), 1).otherwise(0) ).persist()

smr_by_director = matched.groupby("nconst", "is_writer").agg(
    F.sum("is_bad").alias('bad'),
    F.sum("is_good").alias('good'),
    F.count("tconst").alias('total'),
    (F.sum("is_bad")/F.count("tconst")).alias("bad_ratio")
)
smr = smr_by_director.filter(
    (F.col('bad')<=10 ) &
    (F.col('good')>5) &
    (F.col('bad_ratio')<=0.2)
).selectExpr("COUNT(1) AS dir_num", 
             "SUM(is_writer) AS writer", 
             "SUM(is_writer)/COUNT(1) AS pct" 
             ).collect()
writer_num = smr[0].writer
dir_num = smr[0].dir_num


"""Với số đạo diễn đề cử là dir_num, số đạo diễn có kiêm biên kịch là writer_num
Số cách chọn 3 đạo diễn từ writer_num là x = C(writer_num, 3)
Số cách chọn 3 đạo diễn từ dir_num là y = C(dir_num, 3)
Xác suất để 3 đạo diễn là đồng thời là biên kịch là: p = x/y
"""
# / END YOUR CODE

TypeError: ignored

In [ ]:
import math

# def comb(n,r):
#     f = math.factorial
#     return f(n) / f(r) / f(n-r)

# p = math.comb(writer_num, 3)/ math.comb(dir_num,3)

# print(p)
print(writer_num)
print(dir_num)


None
0


### PHẦN II. TỰ CHỌN
[ 10 điểm ] <br>
DS chọn 1 trong 2 câu II.a hoặc II.b
**Chỉ sử dụng dữ liệu train đã được chia sẵn để huấn luyện model, không sử dụng tập test.** <br>
Có thể sử dụng tập test để đánh giá độ chính xác của model.

Điểm số được chấm dựa trên độ chính xác của model trên tập test.

**Câu II.a**. Trên môi trường pyspark, hãy tổng hợp dữ liệu và huấn luyện một mô hình dùng pyspark MLlib dự đoán averageRating

In [ ]:
# CHẠY ĐOẠN CODE SAU | DO NOT MODIFY
train_film = df_basics.filter((F.col("titleType")=="movie") & (F.col("startYear")>=2000) & (F.col("startYear")<=2015) ).select("tconst")
test_film  = df_basics.filter((F.col("titleType")=="movie") & (F.col("startYear")> 2015) & (F.col("startYear")<=2021) ).select("tconst")

train_ratings = df_ratings.join( train_film, on = 'tconst')
test_ratings = df_ratings.join( test_film, on = 'tconst')

In [ ]:
train_ratings.count()

Chỉ sử dụng thông tin của các bộ phim trong train_film (và nhãn là averageRating trong train_ratings) để xây dựng model. <br>
Có thể join thêm với các thông tin ở các bảng khác không giới hạn để bổ sung feature. <br>
Metric đánh giá là MAE

**YÊU CẦU ĐẦU RA** <br>
Dataframe: df_pred {tconst: string, pred: float}, trong đó tconst là các mã bộ phim tương ứng với các bộ phim trong test_film, pred là dự đoán averageRating của model.

In [ ]:
# YOUR CODE START 

#...BUILD YOUR MODEL...

df_pred = 
# / END YOUR CODE

In [ ]:
# df_pred hợp lệ khi testcase báo 2 thông báo PASS như sau:
#>> PASS: dataframe đầu ra đúng định dạng
#>> PASS: dataframe đầu ra đúng số lượng bản ghi
TEST.iia_schema( df_pred )

In [ ]:
# Tính MAE
TEST.mae_iia( df_pred )

**Câu II.b**. Bạn đưa ra giả thuyết thông tin về tiêu đề phim có thể giúp phân loại phim đó có phải là phim không dành cho trẻ nhỏ hay không. <br>
Chỉ sử dụng thông tin từ bảng title.basics.tsv.gz, hãy xây dựng mô hình dự đoán nhãn isAdult. Trong đó, bắt buộc có sử dụng xử lý chuỗi thông tin tiêu đề phim originalTitle.

(Mô hình có thể được xây dựng bằng pyspark hoặc python)

In [ ]:
# CHẠY ĐOẠN CODE SAU | DO NOT MODIFY
train = df_basics.filter((F.col("titleType")=="movie") & (F.col("startYear")>=2000) & (F.col("startYear")<=2015) ).toPandas()
test = df_basics.filter((F.col("titleType")=="movie") & (F.col("startYear")> 2015) & (F.col("startYear")<=2021) ).toPandas()

In [ ]:
train['tconst'].count()

Có thể sử dụng model pyspark hoặc python. <br>
Metric đánh giá là accuracy

**YÊU CẦU ĐẦU RA** <br>
- Mô hình: có các bước biến đổi dữ liệu và xử lý text
- Dữ liệu: pandas dataframe df_pred {tconst: string, pred: int}, trong đó tconst là các mã bộ phim tương ứng với các bộ phim trong test, pred là dự đoán isAdult của model, pred phải mang giá trị 0 hoặc 1.

In [ ]:
# YOUR CODE START 

df_pred = 
# / END YOUR CODE

In [ ]:
# df_pred hợp lệ khi testcase báo 2 thông báo PASS như sau:
#>> PASS: dataframe đầu ra đúng định dạng
#>> PASS: dataframe đầu ra đúng số lượng bản ghi
TEST.iib_schema(df_pred)

In [ ]:
# Tính AUC
TEST.score_iib(df_pred)

**--- YOUR TEST END HERE ---**